## Librerías

In [1]:
# qa_stress_test_parallel.ipynb (Nuevo nombre sugerido para diferenciar)

# --- 0. Configuración Inicial y Carga de Librerías ---
import pandas as pd
import numpy as np
import os
import json
import yaml
import time
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor # Importar para paralelización

# Importación de utils asumiendo que está en la misma carpeta 'notebooks/'
# Asegúrate de que utils.py contenga la clase TimerResult y el @contextmanager def timer(name): yield result
from utils import timer

# Para asegurar que se muestren todas las columnas de Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


# >>>>> RUTA BASE PERSONALIZADA DEL PROYECTO (INTEGRACIÓN DE TU pk_) <<<<<
# Esta ruta es esencial para que el notebook encuentre los archivos
# generados por data_generator.py (data/, company_docs/, etc.)
pk_ = "C:/Users/Alber/OneDrive/Documentos/MADUREZ MLOPS/gerente-relacional-qa-test/"
# Puedes ajustar esta variable si la ubicación de tu proyecto cambia.
# >>>>> FIN DE RUTA BASE PERSONALIZADA <<<<<


## open AI

In [3]:
# --- 1. Cargar Credenciales ---
print("Cargando credenciales de OpenAI...")
try:
    # Usar pk_ para la ruta de credentials.json
    # Asume que credentials.json está en la raíz del proyecto definida por pk_
    with open(os.path.join('credentials.json')) as f: # Citar fuente por el ajuste de ruta
        config_env = json.load(f)
    api_key = config_env["openai_key"]
    client = OpenAI(api_key=api_key)
    print("Credenciales cargadas. Cliente OpenAI inicializado.")
except FileNotFoundError:
    print(f"ERROR: 'credentials.json' no encontrado en la ruta: {os.path.join('credentials.json')}. Asegúrate de crearlo en la raíz de tu proyecto ('{pk_}').")
    api_key = None
    client = None
except KeyError:
    print("ERROR: 'openai_key' no encontrada en 'credentials.json'.")
    api_key = None
    client = None


Cargando credenciales de OpenAI...
Credenciales cargadas. Cliente OpenAI inicializado.


## YAML Prompts

In [4]:
# --- 2. Cargar Prompts del Reporte Unificado ---
print("\nCargando prompts desde prompts.yml...")
# Usar pk_ para la ruta de prompts.yml
prompts_path = os.path.join('prompts.yml') # Citar fuente por el ajuste de ruta
unified_report_prompts = {}
with timer("Carga de prompts y queries") as t_prompts: # Timer para esta etapa
    try:
        with open(prompts_path, 'r', encoding='utf-8') as file:
            all_prompts = yaml.safe_load(file)
        unified_report_prompts = {k: v for k, v in all_prompts.items() if k.startswith('unified_report_')}
        print(f"Prompts de reporte unificado cargados: {list(unified_report_prompts.keys())}")
    except FileNotFoundError:
        print(f"ERROR: '{prompts_path}' no encontrado.")
    except yaml.YAMLError as e:
        print(f"ERROR al parsear YAML: {e}")




Cargando prompts desde prompts.yml...
Prompts de reporte unificado cargados: ['unified_report_1', 'unified_report_2', 'unified_report_3', 'unified_report_4', 'unified_report_5']
[TIMER] Carga de prompts y queries: 0.030s


## Data

In [6]:
# --- 3. Cargar Datas Simuladas (10M Transacciones y 300K Financieros) y Mapeo de Empresas ---
print("\nCargando datas simuladas...")
transactions_df = pd.DataFrame()
financial_df = pd.DataFrame()
company_mapping = [] # Lista de diccionarios {original_name, sanitized_folder_name}

# Decide qué versión de datos cargar (LIGHT o FULL)
USE_LIGHT_DATA = True # Cambia a False para usar los datos grandes (10M/300K)

transactions_file = 'simulated_transactions.csv'
financial_file = 'simulated_financial_metrics.csv'

with timer("Carga de dataframes e historial") as t_data_load: # Timer para esta etapa
    try:
        # Usar pk_ para la ruta de los archivos de datos
        transactions_df = pd.read_csv(os.path.join(pk_, 'data', transactions_file))
        print(f"Data de transacciones cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(transactions_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{transactions_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    try:
        # Usar pk_ para la ruta de los archivos de datos
        financial_df = pd.read_csv(os.path.join(pk_, 'data', financial_file))
        print(f"Data financiera cargada ({'LIGHT' if USE_LIGHT_DATA else 'FULL'}). Registros: {len(financial_df)}")
    except FileNotFoundError:
        print(f"ERROR: '{financial_file}' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")

    # Cargar el mapeo de empresas
    try:
        # Usar pk_ para la ruta del mapeo de empresas
        with open(os.path.join(pk_, 'data', 'company_mapping.json'), 'r', encoding='utf-8') as f:
            company_mapping = json.load(f)
        print(f"Mapeo de {len(company_mapping)} empresas cargado.")
    except FileNotFoundError:
        print(f"ERROR: 'company_mapping.json' no encontrado en la ruta: '{os.path.join(pk_, 'data')}/'. Ejecuta data_generator.py (o _light.py).")
    except json.JSONDecodeError:
        print("ERROR: Fallo al leer 'company_mapping.json'. Asegúrate de que es un JSON válido.")


Cargando datas simuladas...
Data de transacciones cargada (LIGHT). Registros: 2000
Data financiera cargada (LIGHT). Registros: 1000
Mapeo de 25 empresas cargado.
[TIMER] Carga de dataframes e historial: 0.010s


In [7]:
transactions_df.head(1)

,transaction_id,company_name,date,amount,type,description
0,0,Posada-Peña S.A.S.,2024-09-10,586644.59,DEBIT,Libero porro quam modi incidunt.


In [8]:
financial_df.head(1)

,financial_id,company_name,year,revenue,profit,liquidity_ratio,debt_equity_ratio,cash_flow
0,0,Gómez-Cardozo S.A.S.,2021,14103440.43,19772809.19,0.69,2.1,9111115.75


## PDFs

In [22]:
# --- 4. Función para Cargar Contenido de PDFs Simulados por Empresa ---
def load_company_documents(sanitized_folder_name):
    """Carga el contenido textual simulado de los PDFs para una empresa usando su nombre de carpeta sanitizado."""
    doc_contents = {}
    # Usar pk_ para la ruta base de los documentos de las empresas
    base_path = os.path.join(pk_, 'company_docs', sanitized_folder_name)
    if not os.path.exists(base_path):
        print(f"Advertencia: Carpeta de documentos no encontrada para '{sanitized_folder_name}' en '{base_path}'")
        return doc_contents

    with timer(f"Carga documentos para {sanitized_folder_name}") as t_doc_load:
        for doc_type in ['gestion', 'sectorial', 'financiero']:
            file_path = os.path.join(base_path, f'{doc_type}.txt')
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    doc_contents[doc_type] = f.read()
            else:
                doc_contents[doc_type] = "" # Vacío si no se encuentra el archivo
    return doc_contents



## FLOW

In [9]:
# --- 4. Función para Cargar Contenido de PDFs Simulados por Empresa (Tu versión adaptada) ---
def load_company_documents(sanitized_folder_name):
    """Carga el contenido textual simulado de los PDFs para una empresa usando su nombre de carpeta sanitizado."""
    doc_contents = {}
    # Usar pk_ para la ruta base de los documentos de las empresas
    base_path = os.path.join(pk_, 'company_docs', sanitized_folder_name)
    if not os.path.exists(base_path):
        print(f"Advertencia: Carpeta de documentos no encontrada para '{sanitized_folder_name}' en '{base_path}'")
        return doc_contents

    with timer(f"Carga documentos para {sanitized_folder_name}") as t_doc_load:
        for doc_type in ['gestion', 'sectorial', 'financiero']:
            file_path = os.path.join(base_path, f'{doc_type}.txt')
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    doc_contents[doc_type] = f.read()
            else:
                doc_contents[doc_type] = "" # Vacío si no se encuentra el archivo
    return doc_contents

# --- 5. Lógica Principal de Generación de Reporte (Emulación del main.py) ---

def run_unified_report_flow(company_original_name, company_sanitized_folder_name, report_prompt_key, user_query=""):
    """
    Ejecuta un flujo simulado de generación de reporte unificado para una empresa.
    Mide tiempos y recolecta métricas.
    """
    # Asegúrate de que client, unified_report_prompts, company_mapping, transactions_df, financial_df
    # estén definidos globalmente o pasados como argumentos.
    # En este contexto, son globales porque el notebook los carga al principio.
    global client, unified_report_prompts, company_mapping, transactions_df, financial_df

    if client is None or not unified_report_prompts or not company_mapping or transactions_df.empty or financial_df.empty:
        error_msg = "ERROR: Variables globales (client, prompts, mapping, dataframes) no inicializadas. Abortando."
        print(error_msg)
        return {"status": "failed", "error": error_msg}


    print(f"\n--- Ejecutando flujo para {company_original_name} ({report_prompt_key}) ---")
    metrics = {
        "status": "success",
        "total_execution_time": 0.0,
        "llm_input_tokens": 0,
        "llm_output_tokens": 0,
        "data_processed_tx_rows": 0,
        "data_processed_fin_rows": 0,
        "llm_api_latency": 0.0, # Latencia de la llamada al LLM aislada
        "timer_metrics": {} # Para guardar los tiempos de cada [TIMER]
    }
    
    # Timer principal para el flujo completo
    with timer(f"Tiempo total conversación para {company_original_name}") as total_timer_result: # Nombre más específico
        # --- [TIMER] Normalización y decisión de flujo: (SIMULADO)
        with timer("Normalización y decisión de flujo") as t_norm_result:
            time.sleep(0.001) # Pequeña simulación de procesamiento
        metrics["timer_metrics"]["Normalización y decisión de flujo"] = t_norm_result.elapsed_time
        
        # --- [TIMER] Inicialización cliente e historial: (SIMULADO)
        with timer("Inicialización cliente e historial") as t_init_result:
            time.sleep(0.01) # Pequeña simulación
        metrics["timer_metrics"]["Inicialización cliente e historial"] = t_init_result.elapsed_time

        # --- [TIMER] Configuración y modelos: (SIMULADO)
        with timer("Configuración y modelos") as t_config_result:
            time.sleep(0.05) # Simulación de tiempo para configuración
        metrics["timer_metrics"]["Configuración y modelos"] = t_config_result.elapsed_time
        
        # --- [TIMER] Creación de PineconeManagers (incluye carga de docs simulada):
        with timer("Creación de PineconeManagers (carga de documentos)") as t_pinecone_init_result:
            company_docs = load_company_documents(company_sanitized_folder_name)
        metrics["timer_metrics"]["Creación de PineconeManagers (carga de documentos)"] = t_pinecone_init_result.elapsed_time
        
        # --- Preparar el Prompt Final para el LLM ---
        current_report_prompt_template = unified_report_prompts.get(report_prompt_key)
        if not current_report_prompt_template:
            print(f"ERROR: Prompt '{report_prompt_key}' no encontrado.")
            metrics["status"] = "failed"
            return metrics
        
        # --- [TIMER] Filtrado de dataframes por empresa (simulando query a BD) ---
        with timer("Filtrado de dataframes por empresa (simulando query a BD)") as t_filter_df_result:
            company_transactions_df = transactions_df[transactions_df['company_name'] == company_original_name].copy()
            company_financial_df = financial_df[financial_df['company_name'] == company_original_name].copy()
            metrics["data_processed_tx_rows"] = len(company_transactions_df)
            metrics["data_processed_fin_rows"] = len(company_financial_df)
        metrics["timer_metrics"]["Filtrado de dataframes por empresa (simulando query a BD)"] = t_filter_df_result.elapsed_time

        # --- [TIMER] Búsqueda similitud reporte unificado (simulando Pinecone/RAG):
        with timer("Búsqueda similitud reporte unificado") as t_rag_search_result:
            context_from_docs = company_docs.get('gestion', '') + "\n\n" + \
                                company_docs.get('sectorial', '') + "\n\n" + \
                                company_docs.get('financiero', '')
            
            time.sleep(0.01 + len(context_from_docs) / 1000000.0) # Simulación de tiempo RAG
        metrics["timer_metrics"]["Búsqueda similitud reporte unificado"] = t_rag_search_result.elapsed_time

        # --- Rellenar los placeholders del prompt ---
        simulated_products_list = "Préstamos Comerciales, Créditos de Liquidez, Cuentas de Ahorro, CDT."
        simulated_sector = "Tecnología y Servicios Financieros"

        # Asegúrate de que los DFs no estén vacíos antes de to_markdown
        df_desem_pag_cast_md_str = company_transactions_df.head(5).to_markdown(index=False) if not company_transactions_df.empty else "No hay datos de transacciones."
        df_perfilador_md_str = company_financial_df.head(5).to_markdown(index=False) if not company_financial_df.empty else "No hay datos financieros."


        response_va_simulated = "Según Valora Analitik, la empresa ha invertido en IA para optimizar procesos bancarios."
        response_pp_simulated = "En Primera Página se destacó la expansión regional de la empresa en el último año."

        try:
            formatted_prompt = current_report_prompt_template.format(
                company_name=company_original_name,
                user_request=user_query if user_query else f"Genera un reporte unificado para {company_original_name} basado en los datos proporcionados.",
                management_report=company_docs.get('gestion', 'No disponible'),
                sector=simulated_sector,
                sector_report=company_docs.get('sectorial', 'No disponible'),
                financial_report=company_docs.get('financiero', 'No disponible'),
                response_va=response_va_simulated,
                response_pp=response_pp_simulated,
                df_desem_pag_cast_md=df_desem_pag_cast_md_str,
                df_perfilador_md=df_perfilador_md_str,
                products_list=simulated_products_list
            )
        except KeyError as e:
            print(f"ERROR: Placeholder '{e}' no encontrado en el prompt '{report_prompt_key}'. Revisa tu prompts.yml.")
            metrics["status"] = "failed"
            return metrics

        full_messages = [
            {"role": "system", "content": formatted_prompt},
            {"role": "user", "content": user_query if user_query else f"Genera el reporte unificado para {company_original_name}."}
        ]

        # --- [TIMER] Generación prompt + invocación LLM (reporte) ---
        llm_invocation_start_time = time.perf_counter()
        try:
            completion = client.chat.completions.create(
                model="gpt-4", # Asegúrate de tener acceso a este modelo
                temperature=0.0,
                messages=full_messages
            )
            metrics["llm_api_latency"] = time.perf_counter() - llm_invocation_start_time
            
            response_content = completion.choices[0].message.content
            metrics["llm_input_tokens"] = completion.usage.prompt_tokens
            metrics["llm_output_tokens"] = completion.usage.completion_tokens
            
            # Usar la misma métrica de latencia de API para el timer_metrics
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = metrics["llm_api_latency"] 
            
            print(f"Respuesta del LLM generada (primeros 200 chars): {response_content[:200]}...")
        except Exception as e:
            print(f"ERROR en invocación LLM para {company_original_name}: {e}")
            metrics["status"] = "failed"
            metrics["error"] = str(e)
            # Registrar el tiempo del intento incluso si falla
            metrics["timer_metrics"]["Generación prompt + invocación LLM (reporte)"] = time.perf_counter() - llm_invocation_start_time 
            response_content = "ERROR"
    
    # Captura el tiempo total del contexto principal al salir del 'with timer'
    metrics["total_execution_time"] = total_timer_result.elapsed_time

    print(f"\n--- Métricas Finales para {company_original_name} ({report_prompt_key}) ---")
    print(f"Tiempo Total de Ejecución: {metrics['total_execution_time']:.3f}s")
    print(f"Latencia de Invocación LLM (aislada): {metrics['llm_api_latency']:.3f}s")
    print(f"Tokens de Entrada LLM: {metrics['llm_input_tokens']}")
    print(f"Tokens de Salida LLM: {metrics['llm_output_tokens']}")
    print(f"Volumen de Transacciones procesadas: {metrics['data_processed_tx_rows']} filas")
    print(f"Volumen de Financieros procesados: {metrics['data_processed_fin_rows']} filas")
    print(f"Estado del flujo: {metrics['status']}")
    print("Tiempos por subproceso:")
    for k, v in metrics["timer_metrics"].items():
        if v is not None: # Asegurar que el valor no sea None antes de formatear
            print(f"  - {k}: {v:.3f}s")
        else:
            print(f"  - {k}: N/A (tiempo no capturado)")

    return metrics

## Pruebas

In [10]:
company_mapping

[{'original_name': 'Muñoz LLC S.A.S.',
  'sanitized_folder_name': 'empresa_1_Munoz_LLC_SAS'},
 {'original_name': 'Posada-Peña S.A.S.',
  'sanitized_folder_name': 'empresa_2_Posada-Pena_SAS'},
 {'original_name': 'González Inc S.A.S.',
  'sanitized_folder_name': 'empresa_3_Gonzalez_Inc_SAS'},
 {'original_name': 'Gil-Salazar S.A.S.',
  'sanitized_folder_name': 'empresa_4_Gil-Salazar_SAS'},
 {'original_name': 'Galvis, Ramírez and Angarita S.A.S.',
  'sanitized_folder_name': 'empresa_5_Galvis_Ramirez_and_Angarita_SAS'},
 {'original_name': 'Gómez, Sanabria and Pulido S.A.S.',
  'sanitized_folder_name': 'empresa_6_Gomez_Sanabria_and_Pulido_SAS'},
 {'original_name': 'Buitrago, García and García S.A.S.',
  'sanitized_folder_name': 'empresa_7_Buitrago_Garcia_and_Garcia_SAS'},
 {'original_name': 'Reyes-Zambrano S.A.S.',
  'sanitized_folder_name': 'empresa_8_Reyes-Zambrano_SAS'},
 {'original_name': 'Pacheco, Velandia and Ávila S.A.S.',
  'sanitized_folder_name': 'empresa_9_Pacheco_Velandia_and_Avi

In [11]:
unified_report_prompts

{'unified_report_1': 'Eres un analista experto en la elaboración de informes gerenciales para el área comercial del Banco de Bogotá. Tu objetivo, juanto con otros analistas, es generar un informe estratégico comercial (detallado) que busca profundizar relaciones con el cliente {company_name}. Tienes asignado uno de los puntos del informe general.\nPara lograr tu tarea dentro de la división (entre analistas) de objetivo, sigue esta estructura, asegurando calidad analítica y redacción clara. Importante evitar repetir datos entre secciones (especialmente cifras, conclusiones y diagnósticos):\n\n1. Descripción general de la empresa: a partir de toda la información suministrada realiza un resumen que incluya la actividad de la empresa {company_name}; sus principales productos y servicios (realiza una lista que contenga descripciones breves para cada elemento); quiénes son los clientes de la empresa {company_name}; información de los competidores directos de la empresa {company_name} en el m

In [12]:
# --- 6. Ejecutar Pruebas (Ejemplo) ---
# Primero, asegúrate de haber ejecutado data_generator.py (o _light.py)
# para que los archivos CSV, TXT y company_mapping.json existan.

if not company_mapping:
    print("No se pudo cargar el mapeo de empresas. Asegúrate de ejecutar el generador de datos primero.")
else:
    test_company_data = company_mapping[0] # Tomamos la primera empresa del mapeo para la prueba de ejemplo
    test_company_original_name = test_company_data["original_name"]
    test_company_sanitized_folder_name = test_company_data["sanitized_folder_name"]

    print(f"\nRealizando una prueba de ejemplo para la empresa: {test_company_original_name} (Ejecución Secuencial)")
    
    if 'unified_report_1' in unified_report_prompts:
        results = run_unified_report_flow(
            company_original_name=test_company_original_name,
            company_sanitized_folder_name=test_company_sanitized_folder_name,
            report_prompt_key='unified_report_1',
            user_query="Genera el resumen general de la empresa con los datos proporcionados."
        )
        print("\n--- Resultados Detallados de la Prueba de Ejemplo (Secuencial) ---")
        print(json.dumps(results, indent=2))
    else:
        print("El prompt 'unified_report_1' no está disponible en prompts.yml. Por favor, revisa tus prompts.")



Realizando una prueba de ejemplo para la empresa: Muñoz LLC S.A.S. (Ejecución Secuencial)

--- Ejecutando flujo para Muñoz LLC S.A.S. (unified_report_1) ---
[TIMER] Normalización y decisión de flujo: 0.001s
[TIMER] Inicialización cliente e historial: 0.017s
[TIMER] Configuración y modelos: 0.053s
[TIMER] Carga documentos para empresa_1_Munoz_LLC_SAS: 0.004s
[TIMER] Creación de PineconeManagers (carga de documentos): 0.004s
[TIMER] Filtrado de dataframes por empresa (simulando query a BD): 0.002s
[TIMER] Búsqueda similitud reporte unificado: 0.027s
Respuesta del LLM generada (primeros 200 chars): 1. Descripción general de la empresa:

Muñoz LLC S.A.S. es una empresa que ha consolidado su presencia en el mercado delectus, experimentando un crecimiento del 14% en sus operaciones en el último año...
[TIMER] Tiempo total conversación para Muñoz LLC S.A.S.: 13.048s

--- Métricas Finales para Muñoz LLC S.A.S. (unified_report_1) ---
Tiempo Total de Ejecución: 13.048s
Latencia de Invocación LL

## Prueba de estrés

In [13]:
num_companies_to_test = min(5, len(company_mapping)) 
print(num_companies_to_test)

5


In [14]:
# --- Nuevas Pruebas de Estrés con Paralelización ---

all_test_results_parallel = []
# Puedes ajustar cuántas empresas probar (ej: 5, 10, o len(company_mapping) para todas)
# ¡Prueba con un número mayor ahora para ver el efecto de la paralelización!
num_companies_to_test_parallel = min(3, len(company_mapping)) 

# Define el número máximo de llamadas concurrentes
# Este valor es clave para la paralelización. Empieza con 3-5 y ajusta.
MAX_CONCURRENT_LLM_CALLS = 3 

print(f"\nIniciando pruebas de estrés en PARALELO para {num_companies_to_test_parallel} empresas con {MAX_CONCURRENT_LLM_CALLS} llamadas concurrentes...")

# Usamos ThreadPoolExecutor para ejecutar run_unified_report_flow en hilos separados
with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_LLM_CALLS) as executor:
    futures = []
    # Generar las tareas a ejecutar en paralelo
    for i in range(num_companies_to_test_parallel):
        company_data = company_mapping[i]
        company_original_name = company_data["original_name"]
        company_sanitized_folder_name = company_data["sanitized_folder_name"]

        for prompt_key in ['unified_report_1']: # O más prompts si quieres
            if prompt_key in unified_report_prompts:
                print(f"  > Programando tarea para {company_original_name} con prompt: {prompt_key}")
                # 'executor.submit' envía la función para ser ejecutada en un hilo del pool
                future = executor.submit(
                    run_unified_report_flow,
                    company_original_name=company_original_name,
                    company_sanitized_folder_name=company_sanitized_folder_name,
                    report_prompt_key=prompt_key,
                    user_query=f"Genera el reporte de {prompt_key} para {company_original_name} (paralelo)."
                )
                # Guardamos el objeto 'future' junto con los datos de la empresa para poder identificar el resultado
                futures.append((future, company_original_name, prompt_key))
            else:
                print(f"  > Advertencia: El prompt '{prompt_key}' no se encontró para {company_original_name}. Saltando.")

    # Recopilar los resultados a medida que las tareas se completan
    print("\nRecopilando resultados de las tareas paralelas...")
    # Este bucle esperará por cada futuro y recopilará el resultado
    for future, company_name, prompt_key in futures:
        try:
            result = future.result() # Bloquea hasta que el resultado esté disponible
            all_test_results_parallel.append({
                "company_name": company_name,
                "prompt_key": prompt_key,
                "metrics": result
            })
            print(f"  > Tarea completada para {company_name} ({prompt_key}). Estado: {result.get('status', 'N/A')}. Tiempo Total: {result.get('total_execution_time', 0):.3f}s")
        except Exception as exc:
            print(f"  > Tarea para {company_name} ({prompt_key}) generó una excepción: {exc}")
            all_test_results_parallel.append({
                "company_name": company_name,
                "prompt_key": prompt_key,
                "metrics": {"status": "failed", "error": str(exc), "total_execution_time": 0.0} # Incluye un tiempo 0.0 para consistencia
            })

print("\n--- Todas las pruebas de estrés en PARALELO ejecutadas ---")
print("Puedes analizar la variable 'all_test_results_parallel' para ver los resultados consolidados.")
print(json.dumps(all_test_results_parallel, indent=2))


Iniciando pruebas de estrés en PARALELO para 3 empresas con 3 llamadas concurrentes...
  > Programando tarea para Muñoz LLC S.A.S. con prompt: unified_report_1

--- Ejecutando flujo para Muñoz LLC S.A.S. (unified_report_1) ---
  > Programando tarea para Posada-Peña S.A.S. con prompt: unified_report_1

--- Ejecutando flujo para Posada-Peña S.A.S. (unified_report_1) ---
  > Programando tarea para González Inc S.A.S. con prompt: unified_report_1

--- Ejecutando flujo para González Inc S.A.S. (unified_report_1) ---

Recopilando resultados de las tareas paralelas...
[TIMER] Normalización y decisión de flujo: 0.002s
[TIMER] Normalización y decisión de flujo: 0.002s
[TIMER] Normalización y decisión de flujo: 0.003s
[TIMER] Inicialización cliente e historial: 0.018s
[TIMER] Inicialización cliente e historial: 0.017s
[TIMER] Inicialización cliente e historial: 0.018s
[TIMER] Configuración y modelos: 0.064s
[TIMER] Configuración y modelos: 0.064s
[TIMER] Configuración y modelos: 0.064s
[TIMER] 

## Lista de resultados

In [16]:
all_test_results_parallel[0]

{'company_name': 'Muñoz LLC S.A.S.',
 'prompt_key': 'unified_report_1',
 'metrics': {'status': 'success',
  'total_execution_time': 23.109918299996934,
  'llm_input_tokens': 1904,
  'llm_output_tokens': 617,
  'data_processed_tx_rows': 84,
  'data_processed_fin_rows': 48,
  'llm_api_latency': 23.000275699996564,
  'timer_metrics': {'Normalización y decisión de flujo': 0.0024484000023221597,
   'Inicialización cliente e historial': 0.01799349999782862,
   'Configuración y modelos': 0.06397930000093766,
   'Creación de PineconeManagers (carga de documentos)': 0.0021224999945843592,
   'Filtrado de dataframes por empresa (simulando query a BD)': 0.0022028999956091866,
   'Búsqueda similitud reporte unificado': 0.015453899999556597,
   'Generación prompt + invocación LLM (reporte)': 23.000275699996564}}}

In [17]:
all_test_results_parallel[1]

{'company_name': 'Posada-Peña S.A.S.',
 'prompt_key': 'unified_report_1',
 'metrics': {'status': 'success',
  'total_execution_time': 19.5960774999985,
  'llm_input_tokens': 1922,
  'llm_output_tokens': 539,
  'data_processed_tx_rows': 82,
  'data_processed_fin_rows': 33,
  'llm_api_latency': 19.471555799995258,
  'timer_metrics': {'Normalización y decisión de flujo': 0.0015181999988271855,
   'Inicialización cliente e historial': 0.01848720000270987,
   'Configuración y modelos': 0.06389170000329614,
   'Creación de PineconeManagers (carga de documentos)': 0.00594719999935478,
   'Filtrado de dataframes por empresa (simulando query a BD)': 0.0027960000006714836,
   'Búsqueda similitud reporte unificado': 0.027142300001287367,
   'Generación prompt + invocación LLM (reporte)': 19.471555799995258}}}

In [18]:
all_test_results_parallel[2]

{'company_name': 'González Inc S.A.S.',
 'prompt_key': 'unified_report_1',
 'metrics': {'status': 'success',
  'total_execution_time': 23.330686699999205,
  'llm_input_tokens': 1887,
  'llm_output_tokens': 650,
  'data_processed_tx_rows': 78,
  'data_processed_fin_rows': 38,
  'llm_api_latency': 23.21424809999735,
  'timer_metrics': {'Normalización y decisión de flujo': 0.0027689999988069758,
   'Inicialización cliente e historial': 0.016535800001292955,
   'Configuración y modelos': 0.06352340000012191,
   'Creación de PineconeManagers (carga de documentos)': 0.007515700002841186,
   'Filtrado de dataframes por empresa (simulando query a BD)': 0.002793999999994412,
   'Búsqueda similitud reporte unificado': 0.018016900001384784,
   'Generación prompt + invocación LLM (reporte)': 23.21424809999735}}}